# **SETUP**

## Spark UI

In [ ]:
!pip install -q pyspark
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.ui.port', '4050').getOrCreate()
spark

In [ ]:
# Fazer login no site https://dashboard.ngrok.com/get-started/setup para obter autenticação própria
ngrok_token = '27WqUGZ1SkzPl4bQunMXJyasbWZ_5pCgLCsppL7ufk9rzK7j4'

In [ ]:
get_ipython().system_raw(f'ngrok authtoken {ngrok_token}')
get_ipython().system_raw('ngrok http 4050 &')
!sleep 3
print('URL para interface Spark:')
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

URL para interface Spark:


## Libraries

In [ ]:
import pandas as pd
from google.colab import files

import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Load data

In [ ]:
# Load file
_ = files.upload()

Saving vgsales.csv to vgsales (1).csv


## Data Description

This dataset contains records of popular video games in North America, Japan, Europe and other parts of the world. Every video game in this dataset has at least 100k global sales.

[Source](https://www.kaggle.com/datasets/gregorut/videogamesales/code?datasetId=284&sortBy=voteCount) of dataset.

## Data Dictionary

| Column       | Explanation                                               |
|:-------------|:----------------------------------------------------------|
| Rank         | Ranking of overall sales                                  |
| Name         | Name of the game                                          |
| Platform     | Platform of the games release (i.e. PC,PS4, etc.)         |
| Year         | Year the game was released in                             |
| Genre        | Genre of the game                                         |
| Publisher    | Publisher of the game                                     |
| NA_Sales     | Number of sales in North America (in millions)            |
| EU_Sales     | Number of sales in Europe (in millions)                   |
| JP_Sales     | Number of sales in Japan (in millions)                    |
| Other_Sales  | Number of sales in other parts of the world (in millions) |
| Global_Sales | Number of total sales (in millions)                       |

## Data Preview

In [ ]:
# Mostrando os dados com pandas
pd.read_csv('vgsales.csv')

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


# Questões

## Questão 1

Leia os dados com Spark, certificando-se que

1. A tabela tenha duas partições e

2. As colunas da tabela tenham este respectivo schema:

|Column|Data type|
|---|---|
|Rank|integer|
|Name|string|
|Platform|string|
|Year|integer|
|Genre|string|
|Publisher|string|
|NA_Sales|double|
|EU_Sales|double|
|JP_Sales|double|
|Other_Sales|double|
|Global_Sales|double|

In [ ]:
df = spark.read.csv('vgsales.csv', header=True, inferSchema=True)
df = df.withColumn('Year', F.col('Year').cast('int'))
df = df.repartition(2)

In [ ]:
print('Num partitions:', df.rdd.getNumPartitions())
df.printSchema()

Num partitions: 2
root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



## Questão 2

Construa uma tabela com:
- contagem das observações
- média
- desvio padrão
- valor mínimo
- valor máximo

para todas as coluna de vendas (que possuem "Sales" no nome).

**Dica:** uma **única função** consegue calcular todos estes valores.

In [ ]:
sales_columns = [col for col in df.columns if 'Sales' in col]
df.select(*sales_columns).describe().show()

+-------+------------------+-------------------+-------------------+--------------------+------------------+
|summary|          NA_Sales|           EU_Sales|           JP_Sales|         Other_Sales|      Global_Sales|
+-------+------------------+-------------------+-------------------+--------------------+------------------+
|  count|             16598|              16598|              16598|               16598|             16598|
|   mean|0.2646674298108155|0.14665200626581024|0.07778166044101428|0.048063019640918934|0.5374406555006714|
| stddev|0.8166830292988791| 0.5053512312869121| 0.3092906480822032| 0.18858840291271461|1.5550279355699124|
|    min|               0.0|                0.0|                0.0|                 0.0|              0.01|
|    max|             41.49|              29.02|              10.22|               10.57|             82.74|
+-------+------------------+-------------------+-------------------+--------------------+------------------+



## Questão 3

Para cada variável categórica do tipo string, calcule quantas categorias distintas estão presentes.

In [ ]:
string_cols = [field['name'] for field in df.schema.jsonValue()['fields'] if field['type']=='string']
df.select(*[F.count_distinct(col) for col in string_cols]).show()

+--------------------+------------------------+---------------------+-------------------------+
|count(DISTINCT Name)|count(DISTINCT Platform)|count(DISTINCT Genre)|count(DISTINCT Publisher)|
+--------------------+------------------------+---------------------+-------------------------+
|               11493|                      31|                   12|                      579|
+--------------------+------------------------+---------------------+-------------------------+



**Resposta:** A região da Europa e "Outras regiões" tiverem um aumento notável quando comparado ao ínicio da década de 90.

## Questão 4

Qual plataforma vendeu mais, mundialmente, considerando todo o período histórico diponível dos dados?  
Mostre as top 10 plataformas em vendas.

In [ ]:
(
  df
 .groupby('Platform')
 .agg(F.sum('Global_Sales').alias('global_sales_sum'))
 .orderBy(F.desc('global_sales_sum'))
 .limit(10)
 .show()
 )

+--------+------------------+
|Platform|  global_sales_sum|
+--------+------------------+
|     PS2|1255.6399999999956|
|    X360| 979.9600000000003|
|     PS3| 957.8400000000025|
|     Wii| 926.7099999999973|
|      DS| 822.4899999999982|
|      PS| 730.6599999999992|
|     GBA|318.49999999999943|
|     PSP| 296.2799999999996|
|     PS4| 278.0999999999999|
|      PC|258.82000000000085|
+--------+------------------+



## Questão 5

Faça uma tabela com os jogos que aparecem em múltiplas plataformas, ordene de forma que os jogos com mais plataformas apareçam primeiro e responda:
- Qual jogo aparece em mais plataformas? Em quantas plataformas?

In [ ]:
(
 df
 .groupby('Name')
 .agg(F.countDistinct('Platform').alias('n_platform'))
 .where('n_platform>1')
 .orderBy(F.desc('n_platform'))
 .limit(10)
 .collect()
 )

[Row(Name='Need for Speed: Most Wanted', n_platform=10),
 Row(Name='LEGO Marvel Super Heroes', n_platform=9),
 Row(Name='Madden NFL 07', n_platform=9),
 Row(Name='Ratatouille', n_platform=9),
 Row(Name='FIFA 14', n_platform=9),
 Row(Name='Lego Batman 3: Beyond Gotham', n_platform=8),
 Row(Name='Terraria', n_platform=8),
 Row(Name='Monopoly', n_platform=8),
 Row(Name='LEGO Jurassic World', n_platform=8),
 Row(Name='Madden NFL 08', n_platform=8)]

**Resposta:** Need for Speed: Most Wanted. Aparece em 10 plataformas.

## Questão 6

Utilize a API do Pandas no Spark para calcular a soma das vendas globais para cada ano e gênro de jogo. Faça então um gráfico de linhas com os anos no eixo `x`, as vendas no eixo `y`, de forma que cada linha corresponda a um gênero de jogo.

**Dica:** após o cálculo, passar os dados para Pandas antes da plotagem, ou plotar diretamente aproveitando os métodos da classe `pyspark.pandas.DataFrame`.

Analise o gráfico e responda:
- Entre 1980 e 1990, quais gêneros mais venderam?
- Entre 2000 e 2015, quais gêneros mais venderam?

In [ ]:
(
  df
 .pandas_api()
 .groupby(['Year','Genre'])[['Global_Sales']].sum()
 .sort_index()
 .reset_index()
 .plot.line(x='Year', y='Global_Sales', color='Genre')
 .update_layout(title='Genre global sales through time', template='plotly_white')
 )

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


**Resposta:**
- Entre 1980 e 1990 sobressaíram as vendas de plataforma, tiro (shooter) e puzzle.
- Entre 2000 e 2015 teve vendas elevadas de jogos de ação e esporte.

## Questão 7

Registre a tabela usando `createOrReplaceTempView` e faça uso da linguagem SQL criar uma tabela que:
- Considere apenas os anos da década de 90 e 
- Agrupe por ano para responder quantos % cada região teve do total de vendas (vendas globais).

Salve o resultado desta query em uma variável chamada `df_questao7`.

Após isso, execute o seguinte comando `df_questao7.pandas_api().set_index('YEAR').style.background_gradient(cmap='Oranges')` e responda:
- Qual/quais regiões tiveram, relativamente (comparado às vendas globais), mais vendas no fim da década de noventa do que no início?








In [ ]:
df.createOrReplaceTempView('table')

query = """
SELECT
  YEAR,
  ROUND(SUM(NA_Sales) / SUM(Global_Sales) * 100, 2) AS NA_Sales_PCT,
  ROUND(SUM(EU_Sales) / SUM(Global_Sales) * 100, 2) AS EU_Sales_PCT,
  ROUND(SUM(JP_Sales) / SUM(Global_Sales) * 100, 2) AS JP_Sales_PCT,
  ROUND(SUM(Other_Sales) / SUM(Global_Sales) * 100, 2) AS Other_Sales_PCT
FROM
  table
WHERE
  YEAR BETWEEN 1990 AND 1999
GROUP BY
  YEAR
ORDER BY
  YEAR
"""

df_questao7 = spark.sql(query)
df_questao7.show()

+----+------------+------------+------------+---------------+
|YEAR|NA_Sales_PCT|EU_Sales_PCT|JP_Sales_PCT|Other_Sales_PCT|
+----+------------+------------+------------+---------------+
|1990|       51.55|       15.45|       30.13|           2.83|
|1991|       39.59|       12.26|       45.86|            2.3|
|1992|       44.47|       15.38|       37.96|           2.17|
|1993|       32.88|       10.11|       55.09|           1.94|
|1994|       35.56|       18.79|       42.93|           2.78|
|1995|       28.17|       16.91|       51.92|            3.0|
|1996|       43.57|       23.73|       28.84|           3.86|
|1997|       47.14|       24.04|       24.32|           4.54|
|1998|       50.05|       26.08|       19.51|            4.3|
|1999|       50.17|       24.94|       20.83|            4.0|
+----+------------+------------+------------+---------------+



In [ ]:
df_questao7.pandas_api().set_index('YEAR').style.background_gradient(cmap='Oranges')

,NA_Sales_PCT,EU_Sales_PCT,JP_Sales_PCT,Other_Sales_PCT
YEAR,,,,
1990,51.550000,15.450000,30.130000,2.830000
1991,39.590000,12.260000,45.860000,2.300000
1992,44.470000,15.380000,37.960000,2.170000
1993,32.880000,10.110000,55.090000,1.940000
1994,35.560000,18.790000,42.930000,2.780000
1995,28.170000,16.910000,51.920000,3.000000
1996,43.570000,23.730000,28.840000,3.860000
1997,47.140000,24.040000,24.320000,4.540000
1998,50.050000,26.080000,19.510000,4.300000


## Questão 8

Calcule quantas vezes cada região teve vendas superiores às demais.

In [ ]:
regions = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']

(
  df
 .withColumn('max_sale', F.greatest(*regions))
 .withColumn(
     'greatest_sale_region',
     F.when(F.col('NA_Sales')==F.col('max_sale'), 'NA_Sales')
     .when(F.col('EU_Sales')==F.col('max_sale'), 'EU_Sales')
     .when(F.col('JP_Sales')==F.col('max_sale'), 'JP_Sales')
     .otherwise('Other_Sales')
 )
 .groupby('greatest_sale_region')
 .count()
 .show()
)

+--------------------+-----+
|greatest_sale_region|count|
+--------------------+-----+
|            EU_Sales| 2380|
|            JP_Sales| 4029|
|            NA_Sales|10113|
|         Other_Sales|   76|
+--------------------+-----+



## Questão 9

Construe uma tabela que mostre a diferença do total de vendas em um década com a década anterior e responda:
- Qual década apresentou a **menor diferença** comparada à década anterior?

In [ ]:
w = Window.orderBy('Decade')

(
  df
 .where('Year IS NOT NULL')
 .withColumn('Decade', F.floor(F.col('Year')/10)*10) # possível fazer com when, porém mais código necessário
 .groupby('Decade')
 .agg(F.mean('Global_Sales').alias('global_sales_mean'))
 .withColumn('global_sales_mean_lag', F.lag('global_sales_mean').over(w))
 .withColumn('Delta_global_sales_mean', F.col('global_sales_mean') - F.col('global_sales_mean_lag'))
 .orderBy('Decade')
 .show()
)

+------+------------------+---------------------+-----------------------+
|Decade| global_sales_mean|global_sales_mean_lag|Delta_global_sales_mean|
+------+------------------+---------------------+-----------------------+
|  1980|1.8369756097560979|                 null|                   null|
|  1990|0.7229564725833797|   1.8369756097560979|    -1.1140191371727182|
|  2000|0.5043462206776723|   0.7229564725833797|   -0.21861025190570738|
|  2010|0.4899999999999969|   0.5043462206776723|   -0.01434622067767...|
|  2020|              0.29|   0.4899999999999969|    -0.1999999999999969|
+------+------------------+---------------------+-----------------------+



## Questão 10

Utilizando apenas a sintaxe do pyspark, faça uma função que calcule a moda do dataframe para um grupo de colunas que o usuário vai especificar. Além da moda, o dataframe de saída deve conter a quantidade de vezes que a moda aparece, e tambem se é multimodal.

Sua função deve receber como entrada:
- O pyspark dataframe
- A coluna-alvo que desejamos saber a moda e
- Os grupos (colunas do dataframe) que iremos considerar para calcular a moda 

O resultado será uma linha por grupo, contendo a identificação do grupo, a moda, a quantidade de vezes que apareceu, e se é um caso multimodal.

Por exemplo, considerando o seguinte dataframe:

```
df_test = spark.createDataFrame([
  (2.4,'A','A1'), (2.4,'A','A1'), (2.5,'A','A1'), (2.6,'A','A1'), (2.7,'A','A1'),
  (2.4,'B','A1'), (2.5,'B','A1'), (2.5,'B','A1'), (2.6,'B','A1'), (2.4,'B','A1')
], ('values','inner_group', 'main_group')).select('main_group', 'inner_group', 'values')

df_test.show()
```

```
+----------+-----------+------+
|main_group|inner_group|values|
+----------+-----------+------+
|        A1|          A|   2.4|
|        A1|          A|   2.4|
|        A1|          A|   2.5|
|        A1|          A|   2.6|
|        A1|          A|   2.7|
|        A1|          B|   2.4|
|        A1|          B|   2.5|
|        A1|          B|   2.5|
|        A1|          B|   2.6|
|        A1|          B|   2.4|
+----------+-----------+------+
```

Ao aplicar a função:  
`calculate_mode(df=df_test, target_col='values', group_cols=['main_group','inner_group']).show()`

o retorno deve ser:

```
+----------+-----------+----+-----------+----------+
|main_group|inner_group|mode|mode_counts|multimodal|
+----------+-----------+----+-----------+----------+
|        A1|          A| 2.4|          2|     false|
|        A1|          B| 2.4|          2|      true|
+----------+-----------+----+-----------+----------+
```

pois no grupo `A1-A` o valor 2.4 é o que mais aparece (2x), e nenhum outro valor aparece duas vezes também. Já no grupo `A1-B` o valor 2.4 também é o que mais aparece (2X), porém em conjunto com o 2.5, que também aparece duas 2X. Neste caso, devemos reportar apenas uma das modas e informar que é multimodal.

In [ ]:
df_test = spark.createDataFrame([
  (2.4,'A','A1'), (2.4,'A','A1'), (2.5,'A','A1'), (2.6,'A','A1'), (2.7,'A','A1'),
  (2.4,'B','A1'), (2.5,'B','A1'), (2.5,'B','A1'), (2.6,'B','A1'), (2.4,'B','A1')
], ('values','inner_group', 'main_group')).select('main_group', 'inner_group', 'values')

df_test.show()

+----------+-----------+------+
|main_group|inner_group|values|
+----------+-----------+------+
|        A1|          A|   2.4|
|        A1|          A|   2.4|
|        A1|          A|   2.5|
|        A1|          A|   2.6|
|        A1|          A|   2.7|
|        A1|          B|   2.4|
|        A1|          B|   2.5|
|        A1|          B|   2.5|
|        A1|          B|   2.6|
|        A1|          B|   2.4|
+----------+-----------+------+



In [ ]:
def calculate_mode(df, target_col, group_cols=None):
  """Calculate mode

  Parameters
  ----------
  df : pyspark.DataFrame
    The dataframe in which the function will be applied
  target_col : str
    Column name in which the mode will be calculated from
  group_cols : list
    List column names to make the group. If None, no grouping will be considered.

  Returns
  -------
  output: pyspark.DataFrame
    Dataframe with group identification (one per line), the mode, value counts,
    and a column specifying if the group has more than one mode.
  """

  if group_cols is None:
    window_vals = Window.partitionBy('count')
    df_mode = df.groupby(target_col).count()
    max_count = df_mode.select(F.max('count')).collect()[0][0]

    df_mode = df_mode.filter(F.col('count')==max_count)
    nrows = df_mode.count()

    df_mode = df_mode.select(F.col(target_col).alias('mode'), F.col('count').alias('mode_counts'), (F.lit(nrows)>1).alias('multimodal'))

  else:
    window_grp = Window.partitionBy(*group_cols)
    window_grp_vals = Window.partitionBy(*group_cols, target_col)

    df_mode = (
      df
      .withColumn('counts', F.count(F.col(target_col)).over(window_grp_vals))
      .withColumn('counts_max', F.max('counts').over(window_grp))
      .filter(f'counts = counts_max')
      .dropDuplicates(subset=[*group_cols, target_col])
      .withColumn('multimodal', F.count(target_col).over(window_grp)>1)
      .withColumnRenamed(target_col, 'mode')
      .withColumnRenamed('counts_max', 'mode_counts')
      .dropDuplicates(subset=[*group_cols])
      .select(*group_cols, 'mode', 'mode_counts', 'multimodal')
    )
  return df_mode

In [ ]:
calculate_mode(df_test, 'values', ['main_group','inner_group']).show()

+----------+-----------+----+-----------+----------+
|main_group|inner_group|mode|mode_counts|multimodal|
+----------+-----------+----+-----------+----------+
|        A1|          A| 2.4|          2|     false|
|        A1|          B| 2.4|          2|      true|
+----------+-----------+----+-----------+----------+

